In [1]:
%%capture
import scrapy
import re
import dateparser
import json
import os
import pandas as pd
import pickle
import pandas as pd
import spacy
import unicodedata
import nltk
import re
import textacy
import contractions
import itertools
import json
import retinasdk
import math
import nltk
import pyLDAvis.gensim
import numpy as np
from collections import Counter
from datetime import datetime
from nltk import pos_tag
from scrapy.crawler import CrawlerProcess
from gensim import corpora, models
from bs4 import BeautifulSoup
from nltk.tokenize.toktok import ToktokTokenizer
from keras.models import model_from_json
from scipy import spatial
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from nltk import tokenize
from nltk.tokenize import sent_tokenize, word_tokenize
from similarity.jarowinkler import JaroWinkler
import gensim
from gensim.models.word2vec import Word2Vec # the word2vec model gensim class
from gensim.models import KeyedVectors

In [2]:
liteClient = retinasdk.LiteClient("")

In [3]:
# add and remove stopwords
stopwords = ['iphone', 'fitbit', 'alta', 'fit bit', 'hr', 'sony', 'mdr', 'mdr7506', 'product', 'device', 'thing', 'love','daughter','son','husband','wife','mother','father','star','pair','thing','product']
no_stopwords = ['no', 'not']
stopword_list = nltk.corpus.stopwords.words('english')
for stopword in stopwords:
    stopword_list.append(stopword)
for no_stopword in no_stopwords:
    stopword_list.remove(no_stopword)

In [4]:
nlp = spacy.load('en_core_web_lg', parse=False, tag=False, entity=False)
tokenizer = ToktokTokenizer()
def preprocess(review,
               tokenize=False,
               links=False,
               html_tags=False,
               accented_chars=False,
               expand_contractions=False,
               lowercase=False,
               remove_newlines=False,
               isolate_special_characters=False,
               remove_word_types=[],
               remove_numbers=False,
               lemmatize=False,
               remove_special_characters=False,
               remove_extra_whitespace=False,
               remove_stopwords=False):

    # remove links
    if links:
        review = re.sub(
            r'^https?:\/\/.*[\r\n]*', '', review, flags=re.MULTILINE)
    # remove html tags
    if html_tags:
        soup = BeautifulSoup(review, "html.parser")
        review = soup.get_text()
    # remove accented characters
    if accented_chars:
        review = unicodedata.normalize('NFKD', review).encode(
            'ascii', 'ignore').decode('utf-8', 'ignore')
    # expand contractions
    if expand_contractions:
        review = contractions.fix(review)
    # lowercase the text
    if lowercase:
        review = review.lower()
    # remove extra newlines
    if remove_newlines:
        review = re.sub(r'[\r|\n|\r\n]+', ' ', review)
    # insert spaces between special characters to isolate them
    if isolate_special_characters:
        #special_char_pattern = re.compile(r'([{.(-)!}])')
        special_char_pattern = re.compile(r'([{.(-)!}])')
        review = special_char_pattern.sub(" \\1 ", review)
    # lemmatize text
    if lemmatize:
        review = ' '.join([
            str(word.lemma_) if word.lemma_ != '-PRON-' else str(word.text)
            for word in nlp(review)
        ])
    # remove word types
    if remove_word_types:
        tokens = tokenizer.tokenize(review)
        review = ' '.join([
            str(token) for token in tokens
            if nlp(token)[0].pos_ not in remove_word_types
        ])
    # remove numbers
    if remove_numbers:
        review = re.sub('[0-9]', '', review)
    # remove special characters
    if remove_special_characters:
        review = re.sub('[^a-zA-z0-9\s]', '', review)
    # remove extra whitespace
    if remove_extra_whitespace:
        review = re.sub(' +', ' ', review)
    # remove stopwords
    if remove_stopwords:
        review = ' '.join([
            token.strip() for token in tokenizer.tokenize(review)
            if token not in stopword_list
        ])
    if tokenize:
        review = [token.strip() for token in tokenizer.tokenize(review)]

    return review

In [5]:
def contains(string1, string2):
    string1_lemma = preprocess(
        string1[0],
        lemmatize=True)
    string2_lemma = preprocess(
        string2[0],
        lemmatize=True)
    string1_len = len(preprocess(string1_lemma,tokenize=True))
    string2_len = len(preprocess(string2_lemma,tokenize=True))
    keep = []
    if(string1_len>string2_len):
        if string2_lemma in string1_lemma:
            keep.append([string1[0],string1[1],string1[2]])
        else:
            keep.append([string1[0],string1[1],string1[2]])
            keep.append([string2[0],string2[1],string2[2]])
    elif(string2_len>string1_len):
        if string1_lemma in string2_lemma:
            keep.append([string2[0],string2[1],string2[2]])
        else:
            keep.append([string1[0],string1[1],string1[2]])
            keep.append([string2[0],string2[1],string2[2]])
    elif(string1_lemma==string2_lemma):
        keep.append([string1[0],string1[1],string1[2]])
    else:
        keep.append([string1[0],string1[1],string1[2]])
        keep.append([string2[0],string2[1],string2[2]])
    return keep

In [6]:
nlp = spacy.load('en_core_web_lg', parse=False, tag=False, entity=False)
tokenizer = ToktokTokenizer()
sid = SentimentIntensityAnalyzer()
jarowinkler = JaroWinkler()
review_w2v = KeyedVectors.load("data/reviews_300dim_10minwords_10context", mmap='r')
def get_phrase_vec(phrase):
    phrase_vec = np.zeros(300)
    for word in tokenizer.tokenize(phrase):
        phrase_vec += review_w2v.get_vector(word)
    phrase_vec/=len(phrase)
    return phrase_vec
def cosinus_distance(a,b):
    return 1 - spatial.distance.cosine(a,b)

In [7]:
def string_similarity(s1, s2, normalize=True):

    s1_new = preprocess(
        s1,
        lemmatize=True,
        remove_special_characters=True,
        remove_extra_whitespace=True)
    
    s2_new = preprocess(
        s1,
        lemmatize=True,
        remove_special_characters=True,
        remove_extra_whitespace=True)

    return([s2, s1, jarowinkler.similarity(s1_new, s2_new)])



In [8]:
sid = SentimentIntensityAnalyzer()
def extract_entity(s1):
    s1_new = preprocess(
    s1,
    remove_stopwords=True,
    lemmatize=True,
    remove_extra_whitespace=True,
    remove_numbers=True)
    remove_word_types=[
            'PART', 'VERB', 'PRON', 'DET', 'CONJ', 'INTJ', 'NUM',
            'ADP', 'CCONJ', 'ADJ', 'ADV'
        ]
    tokens = tokenizer.tokenize(s1_new)
    s1_new = ' '.join([
        str(token) for token in tokens
        if nlp(token)[0].pos_ not in remove_word_types
    ])

    s1_new = ' '.join([word for word in s1_new.split() if word not in sid.lexicon ])
    return(s1_new)
